# A3C for Kung Fu

## Part 0 - Installing the required packages and importing the libraries

### Installing Gymnasium

In [ ]:
!pip install gymnasium
!pip install "gymnasium[atari, accept-rom-license]"
!apt-get install -y swig
!pip install gymnasium[box2d]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.7/434.7 kB 7.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 13.1 MB/s eta 0:00:00
  Created wheel for AutoROM.accept-rom-license: filename=AutoROM.accept_rom_license-0.6.1-py3-none-any.whl size=446663 sha256=29e1edac72905525168262e3b13b007df329de62eb2cdab4dabcb8244afd7f49
  Stored in directory: /root/.cache/pip/wheels/6b/1b/ef/a43ff1a2f1736d5711faa1ba4c1f61be1131b8899e6a057811
Successfully built AutoROM.accept-rom-license
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  swig4.0
Suggested packages:
  swig-doc swig-examples swig4.0-examples swig4.0-doc
The following NEW packages will be installed

### Importing the libraries

In [ ]:
import cv2
import math
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.multiprocessing as mp
import torch.distributions as distributions
from torch.distributions import Categorical
import gymnasium as gym
from gymnasium.spaces import Box

## Part 1 - Building the AI

### Creating the architecture of the Neural Network

In [ ]:
class Network(nn.Module):

    def __init__(self, action_size):
        super(Network, self).__init__()
        self.fc1 = torch.nn.Linear(128, 256)
        self.fc2a = torch.nn.Linear(256, action_size)
        self.fc2s = torch.nn.Linear(256, 1)

    def forward(self, state):
        x = self.fc1(state)
        x = F.relu(x)
        action_values = self.fc2a(x)
        state_value = self.fc2s(x)
        return action_values, state_value


## Part 2 - Training the AI

### Setting up the environment

In [ ]:
class PreprocessRAM(gym.ObservationWrapper):
    def __init__(self, env):
        super(PreprocessRAM, self).__init__(env)
        self.observation_space = Box(0.0, 1.0, (128,))

    def reset(self):
        obs, info = self.env.reset()
        obs = self.observation(obs)
        return obs, info

    def observation(self, obs):
        return obs.astype(np.float32) / 255.0

def make_env():
    env = gym.make("MsPacman-ram-v4")
    env = PreprocessRAM(env)
    return env

env = make_env()

state_shape = env.observation_space.shape
number_actions = env.action_space.n
print("Observation shape:", state_shape)
print("Number actions:", number_actions)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Observation shape: (128,)
Number actions: 9


### Initializing the hyperparameters

In [ ]:
learning_rate = 1e-4
discount_factor = 0.99
number_environments = 10

### Implementing the A3C class

In [ ]:
class Agent():

    def __init__(self, action_size):
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.action_size = action_size
        self.network = Network(action_size).to(self.device)
        self.optimizer = torch.optim.Adam(self.network.parameters(), lr = learning_rate)

    def act(self, state):
        if state.ndim == 1:
            state = [state]
        state = torch.tensor(state, dtype = torch.float32, device = self.device)
        action_values, _ = self.network(state)
        policy = F.softmax(action_values, dim = -1)
        return np.array([np.random.choice(len(p), p = p) for p in policy.detach().cpu().numpy()])

    def step(self, state, action, reward, next_state, done):
        batch_size = state.shape[0]
        state = torch.tensor(state, dtype = torch.float32, device = self.device)
        next_state = torch.tensor(next_state, dtype = torch.float32, device = self.device)
        reward = torch.tensor(reward, dtype = torch.float32, device = self.device)
        done = torch.tensor(done, dtype = torch.bool, device = self.device).to(dtype = torch.float32)
        action_values, state_value = self.network(state)
        _, next_state_value = self.network(next_state)
        target_state_value = reward + discount_factor * next_state_value * (1 -done)
        advantage = target_state_value - state_value
        probs = F.softmax(action_values, dim = -1)
        logprobs = F.log_softmax(action_values, dim = -1)
        entropy = -torch.sum(probs * logprobs, axis = -1)
        batch_idx = np.arange(batch_size)
        logp_actions = logprobs[batch_idx, action]
        actor_loss = -(logp_actions * advantage.detach()).mean() - 0.001 * entropy.mean()
        critic_loss = F.mse_loss(target_state_value.detach(), state_value)
        total_loss = actor_loss + critic_loss
        self.optimizer.zero_grad()
        total_loss.backward()
        self.optimizer.step()

### Initializing the A3C agent

In [ ]:
agent = Agent(number_actions)

### Evaluating our A3C agent on a single episode

In [ ]:
def evaluate(agent, env, n_episodes = 1000):
    episodes_rewards = []
    for _ in range(n_episodes):
        state, _ = env.reset()
        total_reward = 0
        while True:
            action = agent.act(state)
            state, reward, done, info, _ = env.step(action[0])
            total_reward += reward
            if done:
                break
        episodes_rewards.append(total_reward)
    return episodes_rewards

### Testing multiple agents on multiple environments at the same time

### Training the A3C agent

## Part 3 - Visualizing the results

In [ ]:
import glob
import io
import base64
import imageio
from IPython.display import HTML, display
from gymnasium.wrappers.monitoring.video_recorder import VideoRecorder

def show_video_of_model(agent, env):
    state, _ = env.reset()
    done = False
    frames = []
    while not done:
        frame = env.render()
        frames.append(frame)
        action = agent.act(state)
        state, reward, done, _, _ = env.step(action[0])
    env.close()
    imageio.mimsave('video.mp4', frames, fps=30)

show_video_of_model(agent, env)

def show_video():
    mp4list = glob.glob('*.mp4')
    if len(mp4list) > 0:
        mp4 = mp4list[0]
        video = io.open(mp4, 'r+b').read()
        encoded = base64.b64encode(video)
        display(HTML(data='''<video alt="test" autoplay
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
    else:
        print("Could not find video")

show_video()

Error: Invalid render mode `None`. Supported modes: `human`, `rgb_array`.